## **THEIA@500Hz -- Data processing** ##

In [1]:
#Path of data files
main_path = ''

path_Flowmeter = main_path + ''
path_Watlow = main_path + ''
path_Ulink = main_path + ''
path_P = main_path + ''
path_Picolog = main_path + ''
path_NearField = main_path + '/.csv' #add csv file name
path_FarField = main_path + '/.csv' #add csv file name

#Save plots on your 'MyApp' folder
TGI = '' #TGI of the computer you logged on
TGI_path = 'C:/Users/' + TGI + '/Documents/MyApp'
path_result = TGI_path + '/.html' #Rename the file name as you wish

In [2]:
#Coefficient
coef_UV=116.7
coef_NF=4.27

sampling_rate=5

In [4]:
#Filter parameters 
n=20  #the larger n is, the smoother curve will be
b=[1.0/n]*n
a=1

#Rolling mean parameter
k=100

## Modules importation ##

In [3]:
import pandas as pd
import datetime
from datetime import *
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from scipy.signal import lfilter

## Fonctions ##

In [7]:
###--------------------------------------------------------------------------------------------------------------------------------------------###
#Flowmeter

def extract_data_flowmeter(path):
    '''
    Function to extract data from the Flowmeter.csv file using the 'Date' column.
    * Input: path --> path of the data file 'Flowmeter.csv';
    * Output: pandas table containing the measurement data and the processed date.
    ''' 
    
    filename = path_Flowmeter + '/Flowmeter.csv'

    data = pd.read_csv(filename,
                       delimiter = ',',
                       decimal = '.',
                       usecols = ['Date', 'Time', 'TEMP_SMC1', 'FLOW_SMC1', 'TEMP_SMC2', 'FLOW_SMC2', 'TEMP_SMC3', 'FLOW_SMC3', 'valeur_HT'],
                       header = 2,
                       # on_bad_lines = 'skip',
                       parse_dates = ['Date'],
                       dayfirst = True,
                       infer_datetime_format = True,
                       date_parser = lambda date: pd.to_datetime(date, format = '%d-%m-%Y_%H:%M:%S.%f')
                      )
    data.drop(labels = [data.shape[0]-1, data.shape[0]-2], axis = 0, inplace = True)
    data.set_index(data['Date'], inplace = True)
    data.drop(columns = ['Date', 'Time', 'valeur_HT'], inplace = True)
    
    # for i in range(data.shape[1]):
    #     data[data.columns[i]].rolling(k).mean()[::k]
    #     data[data.columns[i]] = lfilter(b, a, data[data.columns[i]])
    
    data.rename(columns = {'TEMP_SMC1' : 'Bench flowmeter',
                           'FLOW_SMC1' : 'Bench flow',
                           'TEMP_SMC2' : 'Oscillator flowmeter',
                           'FLOW_SMC2' : 'Oscillator flow',
                           'TEMP_SMC3' : 'Amplifier flowmeter',
                           'FLOW_SMC3' : 'Amplifier flow'
                          },
                inplace = True
               )
    
    return data

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Watlow

def extract_data_watlow(path):
    '''
    Function to extract data from the Watlow.csv file using the 'Date' column.
    * Input: path --> path to the data file 'Watlow.csv';
    * Output: pandas table containing the measurement data and the processed date.
    ''' 
    
    filename = path + '/Watlow.csv'

    data = pd.read_csv(filename,
                       delimiter = ',',
                       decimal = '.',
                       usecols = ['Date', 'Time',
                                  'Temperature_SHG', 'Temperature_THG', 'Temperature_Bench',
                                  'Heat_Power_SHG', 'Heat_Power_THG', 'Heat_Power_Bench',
                                  'Cool_Power_SHG', 'Cool_Power_THG', 'Cool_Power_Bench'
                                 ],
                       header = 2,
                       # on_bad_lines = 'skip',
                       parse_dates = ['Date'],
                       dayfirst = True,
                       infer_datetime_format = True,
                       date_parser = lambda date: pd.to_datetime(date, format = '%d-%m-%Y_%H:%M:%S.%f')
                      )
    data.drop(labels = [data.shape[0]-1, data.shape[0]-2], axis = 0, inplace = True)
    data.set_index(data['Date'], inplace = True)
    data.drop(columns = ['Date', 'Time'], inplace = True)
    
    # data['Heat_Power_SHG'].rolling(k).mean()[::k]
    # data['Heat_Power_SHG'] = lfilter(b, a, data['Heat_Power_SHG'])
    data['Cool_Power_SHG'].rolling(k).mean()[::k]
    # data['Cool_Power_SHG'] = lfilter(b, a, data['Cool_Power_SHG'])
    
    # data['Heat_Power_THG'].rolling(k).mean()[::k]
    # data['Heat_Power_THG'] = lfilter(b, a, data['Heat_Power_THG'])
    data['Cool_Power_THG'].rolling(k).mean()[::k]
    # data['Cool_Power_THG'] = lfilter(b, a, data['Cool_Power_THG'])
    
    # data['Heat_Power_Bench'].rolling(k).mean()[::k]
    # data['Heat_Power_Bench'] = lfilter(b, a, data['Heat_Power_Bench'])
    data['Cool_Power_Bench'].rolling(k).mean()[::k]
    # data['Cool_Power_Bench'] = lfilter(b, a, data['Cool_Power_Bench'])
    
    data.drop(columns = ['Heat_Power_SHG', 'Heat_Power_THG', 'Heat_Power_Bench'], inplace = True)
    data.rename(columns = {'Temperature_SHG' : 'Temperature SHG',
                           'Temperature_THG' : 'Temperature THG',
                           'Temperature_Bench' : 'Temperature Bench',
                           'Cool_Power_SHG' : 'Cool Power SHG',
                           'Cool_Power_THG' : 'Cool Power THG',
                           'Cool_Power_Bench' : 'Cool Power Bench'
                          },
                inplace = True
               )
    
    return data

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Picolog

def extact_data_picolog(path):
    '''
    Function to extract data from the file pico.csv using the column 'Date'.
    * Input: path --> path to the data file 'pico.csv';
    * Output: pandas table containing the measurement data and the processed date.
    '''
    
    filename = path + '/picolog.csv'

    data = pd.read_csv(filename,
                       delimiter = ',',
                       decimal = ',',
                       dtype = float,
                       skiprows = 0,
                       index_col = 0,
                       # on_bad_lines = 'skip',
                       parse_dates = True,
                       infer_datetime_format = True,
                       date_parser = lambda date: pd.to_datetime(date, format = '%Y-%m-%dT%H:%M:%S%z')
                      )
    data.columns = data.columns.str.replace('[()]', '')
    data.columns = data.columns.str.replace('Dernier C', '')
    data.index = data.index.tz_localize(None)
    
    for i in range(data.shape[1]):
    #     data[data.columns[i]].rolling(k).mean()[::k]
        data[data.columns[i]] = lfilter(b, a, data[data.columns[i]])
    
    data.rename(columns = {data.columns[0] : 'mirror1',
                           data.columns[1] : 'mirror2',
                           data.columns[2] : 'amp water',
                           data.columns[3] : 'town water',
                           data.columns[4] : 'sensor',
                           data.columns[5] : 'second circuit water',
                           data.columns[6] : 'room air',
                           data.columns[7] : 'room conditionning',
                           data.columns[8] : 'bench resistor near field',
                           data.columns[9] : 'energy meter UV',
                           data.columns[10] : 'tank2',
                           data.columns[11] : 'bench Rmax osc',
                           data.columns[12] : 'dichro1',
                           data.columns[13] : 'dump NC'
                          },
                inplace = True
               )
    
    return data

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Power UV

def extract_data_P(path):
    '''
    Function to extract data from the file P532.csv using the column 'Date'.
    * Input: path --> path to the data file 'P532.csv';
    * Output: pandas table containing the measurement data and the processed date.
    ''' 
    
    filename = path + '/P355.csv'

    data = pd.read_csv(filename,
                       delimiter = '\t',
                       decimal = '.',
                       header = 16,
                       usecols = ['Index', 'Date', 'Second', 'Measurement'],
                       parse_dates = ['Date'],
                       # on_bad_lines = 'skip',
                       dayfirst = True,
                       infer_datetime_format = True,
                       date_parser = lambda date: pd.to_datetime(date, format = '%H:%M:%S, %A, %B %d, %Y')
                      )
    data.set_index(data['Date'], inplace = True)
    data.drop(columns = ['Index', 'Date', 'Second'], inplace = True)
    
    data.query('0 < Measurement < 1000', inplace = True)
    # data['Measurement'].rolling(k).mean()[::k]
    # data['Measurement'] = lfilter(b, a, data['Measurement'])
    
    data.rename(columns = {'Measurement' : 'Power UV'},
                inplace = True
               )
    
    return data

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Energy UV

def extract_data_UV(path):
    '''
    Function to extract data from the file 'Energie UV.csv' using the column 'Date'.
    * Input: path --> path to the data file 'Energie UV.csv';
    * Output: pandas table containing the measurement data and the processed date.
    ''' 
    
    filename = path + '/Energie UV.csv'

    data = pd.read_csv(filename,
                       delimiter = ',',
                       decimal = '.',
                       usecols = ['Date', 'Time', 'Energie_UV'],
                       header = 2,
                       # on_bad_lines = 'skip',
                       parse_dates = ['Date'],
                       dayfirst = True,
                       infer_datetime_format = True,
                       date_parser = lambda date: pd.to_datetime(date, format = '%d-%m-%Y_%H:%M:%S.%f')
                      )
    data.drop(labels = [data.shape[0]-1, data.shape[0]-2], axis = 0, inplace = True)
    data.set_index(data['Date'], inplace = True)
    data.drop(columns = ['Date', 'Time'], inplace = True)
    
    data.query('0 < Energie_UV < 1000', inplace = True)
    data['Energie_UV'] = data['Energie_UV']*coef_UV
    # data['Energie_UV'].rolling(k).mean()[::k]
    # data['Energie_UV'] = lfilter(b, a, data['Energie_UV'])
    
    data.rename(columns = {'Energie_UV' : 'Energy UV'},
                inplace = True
               )
    
    return data

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Energy UV stat

def extract_data_UV_stat(path):
    '''
    Function to extract data from the file Energie UV stat.csv using the column 'Date'.
    * Input: path --> path to the data file 'Energie UV.csv';
    * Output: pandas table containing the measurement data and the processed date.
    ''' 
    
    filename = path + '/Energie UV stat.csv'

    data = pd.read_csv(filename,
                       delimiter = ',',
                       decimal = '.',
                       usecols = ['Date', 'Time', 'Mean', 'Sigma', 'Max', 'Min', 'Rapport'],
                       header = 2,
                       # on_bad_lines = 'skip',
                       parse_dates = ['Date'],
                       dayfirst = True,
                       infer_datetime_format = True,
                       date_parser = lambda date: pd.to_datetime(date, format='%d-%m-%Y_%H:%M:%S.%f')
                      )
    data.drop(labels = [data.shape[0]-1, data.shape[0]-2], axis = 0, inplace = True)
    data.set_index(data['Date'], inplace=True)
    data.drop(columns = ['Date', 'Time'], inplace = True)
    
    data.query('0 < Mean < 1000', inplace = True)
#     # data['Mean'].rolling(k).mean()[::k]
#     # data['Mean'] = lfilter(b, a, data['Mean'])
    
    data.query('0 < Max < 1000', inplace = True)
#     # data['Max'].rolling(k).mean()[::k]
#     # data['Max'] = lfilter(b, a, data['Max'])
    
    data.query('0 < Min < 1000', inplace = True)
#     # data['Min'].rolling(k).mean()[::k]
#     # data['Min'] = lfilter(b, a, data['Min'])
    
#     # data['Rapport'] = (data['Sigma'] / data['Mean'])*100
    data.query('0 < Rapport < 1', inplace = True)
#     # data['Rapport'].rolling(k).mean()[::k]
#     # data['Rapport'] = lfilter(b, a, data['Rapport'])
    
    data.rename(columns = {'Mean' : 'Mean UV',
                           'Sigma' : 'Sigma UV',
                           'Max' : 'Maximum UV',
                           'Min' : 'Minimum UV',
                           'Rapport' : 'RMS UV'
                          },
                inplace = True
               )
    
    return data

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Energy IR

def extract_data_IR(path):
    '''
    Function to extract data from the file 'Energie IR.csv' using the column 'Date'.
    * Input: path --> path to the data file 'Energie IR.csv';
    * Output: pandas table containing the measurement data and the processed date.
    ''' 
    
    filename = path + '/Energie IR.csv'

    data = pd.read_csv(filename,
                       delimiter = ',',
                       decimal = '.',
                       usecols = ['Date', 'Time', 'Energie_IR'],
                       header = 2,
                       # on_bad_lines = 'skip',
                       parse_dates = ['Date'],
                       dayfirst = True,
                       infer_datetime_format = True,
                       date_parser = lambda date: pd.to_datetime(date, format = '%d-%m-%Y_%H:%M:%S.%f')
                      )
    data.drop(labels = [data.shape[0]-1, data.shape[0]-2], axis = 0, inplace = True)
    data.set_index(data['Date'], inplace = True)
    data.drop(columns = ['Date', 'Time'], inplace = True)
    
    data.query('0 < Energie_IR < 5000', inplace = True)    
    # data['Energie_IR'].rolling(k).mean()[::k]
    # data['Energie_IR'] = lfilter(b, a, data['Energie_IR'])
    
    data.rename(columns = {'Energie_IR' : 'Energy IR'},
                inplace = True
               )
    
    return data

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Energy IR stat

def extract_data_IR_stat(path):
    '''
    Function to extract data from the file Energie IR stat.csv using the column 'Date'.
    * Input: path --> path to the data file 'Energie IR.csv';
    * Output: pandas table containing the measurement data and the processed date.
    ''' 
    
    filename = path + '/Energie IR stat.csv'

    data = pd.read_csv(filename,
                       delimiter = ',',
                       decimal = '.',
                       usecols = ['Date', 'Time', 'Mean', 'Sigma', 'Max', 'Min', 'Rapport'],
                       header = 2,
                       # on_bad_lines = 'skip',
                       parse_dates = ['Date'],
                       dayfirst = True,
                       infer_datetime_format = True,
                       date_parser = lambda date: pd.to_datetime(date, format = '%d-%m-%Y_%H:%M:%S.%f')
                      )
    data.drop(labels = [data.shape[0]-1, data.shape[0]-2], axis = 0, inplace = True)
    data.set_index(data['Date'], inplace = True)
    data.drop(columns = ['Date', 'Time'], inplace = True)
    
    data.query('0 < Mean < 5000', inplac = True)
    # data['Mean'].rolling(k).mean()[::k]
    # data['Mean'] = lfilter(b, a, data['Mean'])
    
    data.query('0 < Max < 5000', inplace = True)
    # data['Max'].rolling(k).mean()[::k]
    # data['Max'] = lfilter(b, a, data['Max'])
    
    data.query('0 < Min < 5000', inplace = True)
    # data['Min'].rolling(k).mean()[::k]
    # data['Min'] = lfilter(b, a, data['Min'])
    
    # data['Rapport'] = (data['Sigma'] / data['Mean'])*100
    data.query('0 < Rapport < 5000', inplace = True)
    # data['Rapport'].rolling(k).mean()[::k]
    # data['Rapport'] = lfilter(b, a, data['Rapport'])
    
    data.rename(columns = {'Mean' : 'Mean IR',
                           'Sigma' : 'Sigma IR',
                           'Max' : 'Maximum IR',
                           'Min' : 'Minimum IR',
                           'Rapport' : 'RMS IR'
                          },
                inplace = True
               )
    
    return data

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Depolar power

def extract_data_Puissance_depolar(path):
    '''
    Function to extract data from the file Puissance depolar.csv using the column 'Date'.
    * Input: path --> path to the data file 'Puissance depolar.csv';
    * Output: pandas table containing the measurement data and the processed date.
    ''' 
    
    filename = path + '/Puissance depolar.csv'

    data = pd.read_csv(filename,
                       delimiter = ',',
                       decimal = '.',
                       usecols = ['Date', 'Time', 'Puissance_depolar'],
                       header = 2,
                       # on_bad_lines = 'skip',
                       parse_dates = ['Date'],
                       dayfirst = True,
                       infer_datetime_format = True,
                       date_parser = lambda date: pd.to_datetime(date, format = '%d-%m-%Y_%H:%M:%S.%f')
                      )
    data.drop(labels = [data.shape[0]-1, data.shape[0]-2], axis = 0, inplace = True)
    data.set_index(data['Date'], inplace=True)
    data.drop(columns = ['Date', 'Time'], inplace = True)
    
    data.query('0 < Puissance_depolar < 1000', inplace = True)    
    # data['Puissance_depolar'].rolling(k).mean()[::k]
    # data['Puissance_depolar'] = lfilter(b, a, data['Puissance_depolar'])
    
    data.rename(columns = {'Puissance_depolar' : 'Depolar power'},
                inplace = True
               )
    
    return data

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Depolar power stat

def extract_data_Puissance_depolar_stat(path):
    '''
    Function to extract data from the file Energie IR stat.csv using the column 'Date'.
    * Input: path --> path to the data file 'Energie IR.csv';
    * Output: pandas table containing the measurement data and the processed date.
    ''' 
    
    filename = path + '/Puissance depolar stat.csv'

    data = pd.read_csv(filename,
                       delimiter = ',',
                       decimal = '.',
                       usecols = ['Date', 'Time', 'Mean', 'Sigma', 'Max', 'Min', 'Rapport'],
                       header = 2,
                       # on_bad_lines = 'skip',
                       parse_dates = ['Date'],
                       dayfirst = True,
                       infer_datetime_format = True,
                       date_parser = lambda date: pd.to_datetime(date, format = '%d-%m-%Y_%H:%M:%S.%f')
                      )
    data.drop(labels = [data.shape[0]-1, data.shape[0]-2], axis = 0, inplace = True)
    data.set_index(data['Date'], inplace = True)
    data.drop(columns = ['Date', 'Time'], inplace = True)
    
    data.query('0 < Mean < 1000', inplace = True)
#     # data['Mean'].rolling(k).mean()[::k]
#     data['Mean'] = lfilter(b, a, data['Mean'])

    data.query('0 < Max < 1000', inplace = True)
#     # data['Max'].rolling(k).mean()[::k]
#     data['Max'] = lfilter(b, a, data['Max'])

    data.query('0 < Min < 1000', inplace = True)
#     # data['Min'].rolling(k).mean()[::k]
#     data['Min'] = lfilter(b, a, data['Min'])

#     # data['Rapport'] = (data['Sigma'] / data['Mean'])*100
    data.query('0 < Rapport < 1000', inplace = True)
#     # data['Rapport'].rolling(k).mean()[::k]
#     data['Rapport'] = lfilter(b, a, data['Rapport'])
    
    data.rename(columns = {'Mean' : 'Mean Pd',
                           'Sigma' : 'Sigma Pd',
                           'Max' : 'Maximum Pd',
                           'Min' : 'Minimum Pd',
                           'Rapport' : 'RMS Pd'
                          },
                inplace = True
               )
    
    return data

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Camera Far Field

def extract_data_cameraFF(path):
    '''
    Function to extract data from a camera file using the 'Date' column.
    * Input: path --> path to the data file;
    * Output: pandas table containing the measurement data and the processed date.
    '''
    
    data = pd.read_csv(path,
                       delimiter = ',',
                       decimal = '.',
                       usecols = ['Date', 'Time', 'Position_x', 'Position_y', 'Ray_x', 'Ray_y', 'X_width', 'Y_width', 'Fit_Order_X', 'Fit_Order_Y', 'Steepness_REX', 'Steepness_FEX', 'Steepness_REY', 'Steepness_FEY', 'Peak_Energy_Density', 'Ellipticity'],
                       # on_bad_lines = 'skip',
                       parse_dates = ['Time'],
                       dayfirst = True,
                       infer_datetime_format = True,
                       date_parser = lambda date: pd.to_datetime(date, format = '%d-%m-%Y_%H:%M:%S.%f'),
                       engine = 'python'
                      )
    data.set_index(data['Date'], inplace=True)
    data.drop(columns = ['Date', 'Time', 'Ray_x', 'Ray_y', 'Fit_Order_X', 'Fit_Order_Y', 'Steepness_REX', 'Steepness_FEX', 'Steepness_REY', 'Steepness_FEY', 'Peak_Energy_Density', 'Ellipticity'], inplace = True)
    
    data['Position_x'] = data['Position_x'] - data['Position_x'].mean()
    data['Position_y'] = data['Position_y'] - data['Position_y'].mean()
    
    data.query('-1000 < Position_x < 1000', inplace = True)
    # data['Position_x'].rolling(k).mean()[::k]
    data['Position_x'] = lfilter(b, a, data['Position_x'])
    
    data.query('-1000 < Position_y < 1000', inplace = True)
    # data['Position_y'].rolling(k).mean()[::k]
    data['Position_y'] = lfilter(b, a, data['Position_y'])
    
    data.query('0 < X_width < 1000', inplace = True)
#     data['X_width'].rolling(k).mean()[::k]
    data['X_width'] = lfilter(b, a, data['X_width'])
    
    data.query('0 < Y_width < 1000', inplace = True)
#     data['Y_width'].rolling(k).mean()[::k]
    data['Y_width'] = lfilter(b, a, data['Y_width'])
    
    data.rename(columns = {'Position_x' : 'Position x FF',
                           'Position_y' : 'Position y FF',
                           'X_width' : '1/e^2 X width FF',
                           'Y_width' : '1/e^2 Y width FF'
                          },
                inplace = True
               )
    
    return data

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Camera Near Field

def extract_data_cameraNF(path):
    '''
    Function to extract data from a camera file using the 'Date' column.
    * Input: path --> path to the data file;
    * Output: pandas table containing the measurement data and the processed date.
    '''
    
    data = pd.read_csv(path,
                       delimiter = ',',
                       decimal = '.',
                       usecols = ['Date', 'Time', 'Position_x', 'Position_y', 'Ray_x', 'Ray_y', 'X_width', 'Y_width', 'Fit_Order_X', 'Fit_Order_Y', 'Steepness_REX', 'Steepness_FEX', 'Steepness_REY', 'Steepness_FEY', 'Peak_Energy_Density', 'Ellipticity'],
                       # on_bad_lines = 'skip',
                       parse_dates = ['Date'],
                       dayfirst = True,
                       infer_datetime_format = True,
                       date_parser = lambda date: pd.to_datetime(date, format = '%d-%m-%Y_%H:%M:%S.%f'),
                       engine = 'python'
                      )
    data.set_index(data['Date'], inplace=True)
    data.drop(columns = ['Date', 'Time', 'Ray_x', 'Ray_y', 'Fit_Order_X', 'Fit_Order_Y', 'Steepness_REX', 'Steepness_FEX', 'Steepness_REY', 'Steepness_FEY', 'Peak_Energy_Density', 'Ellipticity'], inplace = True)
    
    data['Position_x'] = data['Position_x']*coef_NF
    data['Position_y'] = data['Position_y']*coef_NF
    data['X_width'] = (data['X_width']*coef_NF)/1000
    data['Y_width'] = (data['Y_width']*coef_NF)/1000
    
    data['Position_x'] = data['Position_x'] - data['Position_x'].mean()
    data['Position_y'] = data['Position_y'] - data['Position_y'].mean()
    
    data.query('-1000 < Position_x < 1000', inplace = True)
    # data['Position_x'].rolling(k).mean()[::k]
    data['Position_x'] = lfilter(b, a, data['Position_x'])
    
    data.query('-1000 < Position_y < 1000', inplace = True)
    # data['Position_y'].rolling(k).mean()[::k]
    data['Position_y'] = lfilter(b, a, data['Position_y'])
    
    data.query('0 < X_width < 100', inplace = True)
#     data['X_width'].rolling(k).mean()[::k]
    data['X_width'] = lfilter(b, a, data['X_width'])
    
    data.query('0 < Y_width < 100', inplace = True)
#     data['Y_width'].rolling(k).mean()[::k]
    data['Y_width'] = lfilter(b, a, data['Y_width'])
    
    data.rename(columns = {'Position_x' : 'Position x NF',
                           'Position_y' : 'Position y NF',
                           'X_width' : '1/e^2 X width NF',
                           'Y_width' : '1/e^2 Y width NF'
                          },
                inplace = True
               )
    
    return data

## Affichage ##

In [21]:
###--------------------------------------------------------------------------------------------------------------------------------------------###
#Flowmeter (Temperatures) + Watlow + Picolog

def plot_temperature(df_flow_tmp, df_watlow, df_pico):
    '''
    Function to plot the results of the measurements according to the date.
    * Input: dataframe -> pandas table containing the data of measurements and the processed date;
    * Output: displays directly a graph with the measurements in ordinate and the date in abscissa.
    '''
    
    yd = [[0.86, 1],
          [0.66, 0.85],
          [0.5, 0.65],
          [0.33, 0.49],
          [0.16, 0.32],
          [0, 0.15]
         ]
    w = [0, 0.05, 0.3, 0.7, 0.95, 1]
    
    fig = go.Figure()
    
    fig.update_layout(title_x=0.5, 
                      title_text='Temperature', 
                      showlegend=True,
                      legend=dict(groupclick='toggleitem'),
                      legend_title_text='Legend',
                      xaxis=dict(title='Date'),
                      yaxis=dict(title='Temperature (°C)',
                                 range=[10,35]
                                )
                     )
    
    #Flowmeter
    for i in range(len(df_flow_tmp.columns)):
        fig.add_trace(go.Scatter(x=df_flow_tmp.index,
                                 y=df_flow_tmp[df_flow_tmp.columns[i]],
                                 name=('%s' %(df_flow_tmp.columns[i])),
                                 xaxis='x',
                                 yaxis='y',
                                 visible=True,
                                 legendgroup='Temperature',
                                 legendgrouptitle_text='Temperature'
                                )
                     )
    
    #Watlow
    for i in range(len(df_watlow.columns)):
        fig.add_trace(go.Scatter(x=df_watlow.index,
                                 y=df_watlow[df_watlow.columns[i]],
                                 name=('%s' %(df_watlow.columns[i])),
                                 xaxis='x',
                                 yaxis='y',
                                 visible=True,
                                 legendgroup='Temperature'
                                )
                     )
    
    #Picolog
    for i in range(len(df_pico.columns)):
        fig.add_trace(go.Scatter(x=df_pico.index,
                                 y=df_pico[df_pico.columns[i]],
                                 name=('%s' %(df_pico.columns[i])),
                                 xaxis='x',
                                 yaxis='y',
                                 visible='legendonly',
                                 legendgroup='Temperature'
                                )
                     )
     
    # fig.show()
    fig.write_html(TGI_path + '/temperature.html', 
                   auto_open=True,
                   config={'modeBarButtonsToAdd':['drawline',
                                                  'drawopenpath',
                                                  'drawclosedpath',
                                                  'drawcircle',
                                                  'drawrect',
                                                  'eraseshape'
                                                 ]
                          }
                  )
    
    return 1

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Flow

def plot_flow(df_flow):
    '''
    Function to plot the results of the measurements according to the date.
    * Input: dataframe -> pandas table containing the data of measurements and the processed date;
    * Output: displays directly a graph with the measurements in ordinate and the date in abscissa.
    '''
    
    yd = [[0.86, 1],
          [0.66, 0.85],
          [0.5, 0.65],
          [0.33, 0.49],
          [0.16, 0.32],
          [0, 0.15]
         ]
    w = [0, 0.05, 0.3, 0.7, 0.95, 1]
    
    fig = go.Figure()
    
    fig.update_layout(title_x=0.5, 
                      title_text='Flow',
                      xaxis=dict(title='Date'),
                      yaxis=dict(title='Flow',
                                 range=[0,10]
                                ),
                      showlegend=True,
                      legend_title_text='Sensors'
                     )
    
    for i in range(len(df_flow.columns)):
        fig.add_trace(go.Scatter(x=df_flow.index,
                                 y=df_flow[df_flow.columns[i]],
                                 name=df_flow.columns[i]
                                )
                     )
    
    # fig.show()
    fig.write_html(TGI_path + '/flow.html', 
                   auto_open=True,
                   config={'modeBarButtonsToAdd':['drawline',
                                                  'drawopenpath',
                                                  'drawclosedpath',
                                                  'drawcircle',
                                                  'drawrect',
                                                  'eraseshape'
                                                 ]
                          }
                  )
    
    return 1

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Energy UV + RMS + Power UV

def plot_UV(df_UV, df_UV_RMS, df_UV_hide, df_P):
    '''
    Function to plot the results of the measurements according to the date.
    * Input: dataframe -> pandas table containing the data of measurements and the processed date;
    * Output: displays directly a graph with the measurements in ordinate and the date in abscissa.
    '''
    
    yd = [[0.86, 1],
          [0.66, 0.85],
          [0.5, 0.65],
          [0.33, 0.49],
          [0.16, 0.32],
          [0, 0.15]
         ]
    w = [0, 0.05, 0.3, 0.7, 0.95, 1]
    
    color_UV=['#19D3F3']
    color_UV_RMS=['#636EFA']
    color_UV_hide=['#8C564B', '#222A2A', '#3366CC']
    color_P=['#AB63FA']
    
    fig = go.Figure()
    
    fig.update_layout(title_x=0.5,
                      title_text='UV', 
                      showlegend=True,
                      legend=dict(groupclick='toggleitem'),
                      legend_title_text='Legend',
                      xaxis=dict(domain=[w[1],w[-2]],
                                title='Date'),
                      yaxis=dict(title='Energy (mJ)',
                                 titlefont=dict(color=color_UV[0]),
                                 tickfont=dict(color=color_UV[0]),
                                 side='left',
                                 range=[400,600]
                                ),
                      yaxis2=dict(title='RMS (%)',
                                  titlefont=dict(color=color_UV_RMS[0]),
                                  tickfont=dict(color=color_UV_RMS[0]),
                                  anchor='free',
                                  overlaying='y',
                                  side='left',
                                  position=w[0],
                                  range=[0,0.5]
                                 ),
                      yaxis3=dict(title='Power (W)',
                                  titlefont=dict(color=color_P[0]),
                                  tickfont=dict(color=color_P[0]), 
                                  anchor='x',
                                  overlaying='y',
                                  side='right',
                                  range=[200,300]
                                 )
                     )
    
    for i in range(len(df_UV.columns)):
        fig.add_trace(go.Scatter(x=df_UV.index,
                                 y=df_UV[df_UV.columns[i]],
                                 name=df_UV.columns[i],
                                 line=dict(color=color_UV[i]),
                                 legendgroup='Energy',
                                 legendgrouptitle_text='Energy',
                                 yaxis='y1'
                                )
                     )

    fig.add_trace(go.Scatter(x=df_UV_RMS.index,
                             y=df_UV_RMS[df_UV_RMS.columns[0]],
                             name=df_UV_RMS.columns[0],
                             line=dict(color=color_UV_RMS[i]),
                             legendgroup='Energy',
                             yaxis='y2'
                            )
                 )
    
    for i in range(len(df_UV_hide.columns)):
        fig.add_trace(go.Scatter(x=df_UV_hide.index,
                                 y=df_UV_hide[df_UV_hide.columns[i]],
                                 name=df_UV_hide.columns[i],
                                 line=dict(color=color_UV_hide[i]),
                                 visible='legendonly',
                                 legendgroup='Energy',
                                 yaxis='y1'
                                )
                     )
    
    for i in range(len(df_P.columns)):
        fig.add_trace(go.Scatter(x=df_P.index,
                                 y=df_P[df_P.columns[i]],
                                 name=df_P.columns[i],
                                 line=dict(color=color_P[i]),
                                 legendgroup='Power',
                                 legendgrouptitle_text='Power',
                                 yaxis='y3'
                                )
                     )
    
    # fig.show()
    fig.write_html(TGI_path + '/UV.html', 
                   auto_open=True,
                   config={'modeBarButtonsToAdd':['drawline',
                                                  'drawopenpath',
                                                  'drawclosedpath',
                                                  'drawcircle',
                                                  'drawrect',
                                                  'eraseshape'
                                                 ]
                          }
                  )
    
    return 1

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Energy IR + RMS + Depolar power

def plot_IR(df_IR, df_IR_RMS, df_IR_hide, df_Pd):
    '''
    Function to plot the results of the measurements according to the date.
    * Input: dataframe -> pandas table containing the data of measurements and the processed date;
    * Output: displays directly a graph with the measurements in ordinate and the date in abscissa.
    '''
    
    yd = [[0.86, 1],
          [0.66, 0.85],
          [0.5, 0.65],
          [0.33, 0.49],
          [0.16, 0.32],
          [0, 0.15]
         ]
    w = [0, 0.05, 0.3, 0.7, 0.95, 1]
    
    color_IR=['#EF553B']
    color_RMS=['#FFA15A']
    color_IR_hide=['#8C564B', '#222A2A', '#3366CC']
    color_Pd=['#636EFA']
    
    fig = go.Figure()
    
    fig.update_layout(title_x=0.5,
                      title_text='IR', 
                      showlegend=True,
                      legend=dict(groupclick='toggleitem'),
                      legend_title_text='Legend',
                      xaxis=dict(domain=[w[1],w[-2]],
                                 title='Date'
                                ),
                      yaxis=dict(title='Energy (mJ)',
                                 titlefont=dict(color=color_IR[0]),
                                 tickfont=dict(color=color_IR[0]),
                                 side='left'
                                ),
                      yaxis2=dict(title='RMS (%)',
                                  titlefont=dict(color=color_RMS[0]),
                                  tickfont=dict(color=color_RMS[0]),
                                  anchor='free',
                                  overlaying='y',
                                  side='left',
                                  position=w[0],
                                  range=[0,0.2]
                                 ),
                      yaxis3=dict(title='Power (W)',
                                  titlefont=dict(color=color_Pd[0]),
                                  tickfont=dict(color=color_Pd[0]), 
                                  anchor='x',
                                  overlaying='y',
                                  side='right',
                                  range=[30,50]
                                 )
                     )
    
    for i in range(len(df_IR.columns)):
        fig.add_trace(go.Scatter(x=df_IR.index,
                                 y=df_IR[df_IR.columns[i]],
                                 name=df_IR.columns[i],
                                 line=dict(color=color_IR[i]),
                                 legendgroup='Energy',
                                 legendgrouptitle_text='Energy',
                                 yaxis='y1'
                                )
                     )

    fig.add_trace(go.Scatter(x=df_IR_RMS.index,
                             y=df_IR_RMS[df_IR_RMS.columns[0]],
                             name=df_IR_RMS.columns[0],
                             line=dict(color=color_RMS[0]),
                             legendgroup='Energy',
                             yaxis='y2'
                            )
                 )
    
    for i in range(len(df_IR_hide.columns)):
        fig.add_trace(go.Scatter(x=df_IR_hide.index,
                                 y=df_IR_hide[df_IR_hide.columns[i]],
                                 name=df_IR_hide.columns[i],
                                 line=dict(color=color_IR_hide[i]),
                                 visible='legendonly',
                                 legendgroup='Energy',
                                 yaxis='y1'
                                )
                     )
    
    for i in range(len(df_Pd.columns)):
        fig.add_trace(go.Scatter(x=df_Pd.index,
                                 y=df_Pd[df_Pd.columns[i]],
                                 name=df_Pd.columns[i],
                                 line=dict(color=color_Pd[i]),
                                 legendgroup='Power',
                                 legendgrouptitle_text='Power',
                                 yaxis='y3'
                                )
                     )
    
    # fig.show()
    fig.write_html(TGI_path + '/IR.html', 
                   auto_open=True,
                   config={'modeBarButtonsToAdd':['drawline',
                                                  'drawopenpath',
                                                  'drawclosedpath',
                                                  'drawcircle',
                                                  'drawrect',
                                                  'eraseshape'
                                                 ]
                          }
                  )
    
    return 1

###--------------------------------------------------------------------------------------------------------------------------------------------###
#Camera - Near Field/Far Field

def plot_field(df_field, field):
    '''
    Function to plot the results of the measurements according to the date.
    * Input: dataframe -> pandas table containing the data of measurements and the processed date;
    * Output: displays directly a graph with the measurements in ordinate and the date in abscissa.
    '''
    
    if (field == 'Near'):
        ff_nf = 'NF'
        width = 'mm'
    else:
        ff_nf = 'FF'
        width = 'µm'
        
    yd = [[0.86, 1],
          [0.66, 0.85],
          [0.5, 0.65],
          [0.33, 0.49],
          [0.16, 0.32],
          [0, 0.15]
         ]
    w = [0, 0.05, 0.3, 0.7, 0.95, 1]
    
    color_Position=['#19D3F3', '#636EFA']
    color_Width=['#FFA15A', '#EF553B']
    
    fig = go.Figure()
    
    fig.update_layout(title_x=0.5,
                      title_text=('Camera - %s Field' %field), 
                      showlegend=True,
                      legend=dict(groupclick='toggleitem'),
                      legend_title_text='Legend',
                      xaxis=dict(domain=[w[1],w[-2]],
                                 title='Date'
                                ),
                      yaxis=dict(title='Position (µm)',
                                 side='left'
                                ),
                      yaxis2=dict(title='Width (%s)' %width,
                                  anchor='x',
                                  overlaying='y',
                                  side='right'
                                 )
                     )
    
    #Position x
    fig.add_trace(go.Scatter(x=df_field.index, 
                             y=df_field['Position x %s' %ff_nf], 
                             name='Position x',
                             xaxis='x',
                             yaxis='y1',
                             line=dict(color=color_Position[0]),
                             visible=True,
                             legendgroup='Position',
                             legendgrouptitle_text='Position',
                            )
                 )
    
    #Position y
    fig.add_trace(go.Scatter(x=df_field.index, 
                             y=df_field['Position y %s' %ff_nf], 
                             name='Position y',
                             xaxis='x',
                             yaxis='y1',
                             line=dict(color=color_Position[1]),
                             visible=True,
                             legendgroup='Position',
                            )
                 )
    
    #Width x
    fig.add_trace(go.Scatter(x=df_field.index, 
                             y=df_field['1/e^2 X width %s' %ff_nf], 
                             name='1/e^2 x',
                             xaxis='x',
                             yaxis='y2',
                             line=dict(color=color_Width[0]),
                             visible=True,
                             legendgroup='Width',
                             legendgrouptitle_text='Width'
                            )
                 )
    
    #Width y
    fig.add_trace(go.Scatter(x=df_field.index, 
                             y=df_field['1/e^2 Y width %s' %ff_nf], 
                             name='1/e^2 y',
                             xaxis='x',
                             yaxis='y2',
                             line=dict(color=color_Width[1]),
                             visible=True,
                             legendgroup='Width'
                            )
                 )
    
    # fig.show()
    fig.write_html(TGI_path + '/' + field + '_field.html', 
                   auto_open=True,
                   config={'modeBarButtonsToAdd':['drawline',
                                                  'drawopenpath',
                                                  'drawclosedpath',
                                                  'drawcircle',
                                                  'drawrect',
                                                  'eraseshape'
                                                 ]
                          }
                  )
    
    return 1

In [22]:
def plot_xshared(dfs):
    '''
    Function to plot the results of the measurements according to the date.
    * Input: dataframes -> pandas table containing the data of measurements and the processed date;
    * Output: displays directly a graph with the measurements in ordinate and the common date in abscissa.
    '''
        
    df_flow_tmp, df_watlow, df_pico, df_flow, df_UV, df_UV_RMS, df_UV_hide, df_P, df_IR, df_IR_RMS, df_IR_hide, df_Pd, df_nearfield, df_farfield = dfs
    
    yd = [[0.86, 1],
          [0.66, 0.85],
          [0.5, 0.65],
          [0.33, 0.49],
          [0.16, 0.32],
          [0, 0.15]
         ]
    w = [0, 0.05, 0.3, 0.7, 0.95, 1]
    
    color_UV=['#19D3F3']
    color_UV_RMS=['#636EFA']
    color_UV_hide=['#8C564B', '#222A2A', '#3366CC']
    color_P=['#AB63FA']
    
    color_IR=['#EF553B']
    color_IR_RMS=['#FFA15A']
    color_IR_hide=['#8C564B', '#222A2A', '#3366CC']
    color_Pd=['#636EFA']
    
    color_Position=['#19D3F3', '#636EFA']
    color_Width=['#FFA15A', '#EF553B']
    
    fig = go.Figure()
    
    fig.update_layout(showlegend=True,
                      title_text=('Endurance THEIA@500Hz from %s to %s' %(df_UV.index[1].strftime('%d-%m-%Y %H:%M:%S'), df_UV.index[-1].strftime('%d-%m-%Y %H:%M:%S'))),
                      title_x=0.5,
                      legend=dict(groupclick='toggleitem'),
                      legend_title_text='Legend',
                      xaxis1=dict(anchor='y1',
                                  domain=[w[1],w[-2]],
                                  matches='x12',
                                  showticklabels=False
                                 ),
                      yaxis1=dict(title='Energy UV (mJ)',
                                  titlefont=dict(color=color_UV[0]),
                                  tickfont=dict(color=color_UV[0]),
                                  domain=yd[0],
                                  range=[400,600],
                                  showgrid=False
                                 ), #UV
                      yaxis2=dict(title='RMS UV (%)',
                                  titlefont=dict(color=color_UV_RMS[0]),
                                  tickfont=dict(color=color_UV_RMS[0]),
                                  anchor='free',
                                  overlaying='y1',
                                  matches=None,
                                  side='left',
                                  range=[0,0.5],
                                  position=w[0],
                                  domain=yd[0],
                                  showgrid=False
                                 ), #UV
                      yaxis3=dict(title='Power UV (W)',
                                  titlefont=dict(color=color_P[0]),
                                  tickfont=dict(color=color_P[0]),
                                  anchor='x1',
                                  overlaying='y1',
                                  matches=None,
                                  side='right',
                                  range=[200,300],
                                  domain=yd[0],
                                  showgrid=False
                                 ), #UV
                      xaxis4=dict(anchor='y4',
                                  matches='x12',
                                  showticklabels=False,
                                  domain=[w[1],w[-2]]
                                 ),
                      yaxis4=dict(title='Energy IR (mJ)',
                                  titlefont=dict(color=color_IR[0]),
                                  tickfont=dict(color=color_IR[0]),
                                  domain=yd[1],
                                  showgrid=False
                                 ), #IR
                      yaxis5=dict(title='RMS IR (%)',
                                  titlefont=dict(color=color_IR_RMS[0]),
                                  tickfont=dict(color=color_IR_RMS[0]),
                                  anchor='free',
                                  overlaying='y4',
                                  matches=None,
                                  side='left',
                                  range=[0,0.2],
                                  position=w[0],
                                  domain=yd[1],
                                  showgrid=False
                                 ), #IR
                      yaxis6=dict(title='Power IR (W)',
                                  titlefont=dict(color=color_Pd[0]),
                                  tickfont=dict(color=color_Pd[0]),
                                  anchor='x6',
                                  overlaying='y4',
                                  matches=None,
                                  side='right',
                                  range=[30,50],
                                  domain=yd[1],
                                  showgrid=False
                                 ), #IR
                      xaxis7=dict(anchor='y7',
                                  matches='x12',
                                  showticklabels=False,
                                  domain=[w[1],w[-2]]
                                 ),
                      yaxis7=dict(title='Temperature (°C)',
                                  anchor='x7',
                                  overlaying='y7',
                                  matches=None,
                                  side='left',
                                  range=[10,35],
                                  domain=yd[2],
                                  showgrid=False
                                 ), #temperature
                      xaxis8=dict(anchor='y8',
                                  matches='x12',
                                  showticklabels=False,
                                  domain=[w[1],w[-2]]
                                 ),
                      yaxis8=dict(title='Position FF (µm)',
                                  titlefont=dict(color=color_Position[0]),
                                  tickfont=dict(color=color_Position[0]),
                                  domain=yd[3],
                                  range=[-200,200],
                                  showgrid=False
                                 ), #far field
                      yaxis9=dict(title='Width FF (µm)',
                                  titlefont=dict(color=color_Width[0]),
                                  tickfont=dict(color=color_Width[0]),
                                  anchor='x9',
                                  overlaying='y8',
                                  matches=None,
                                  side='right',
                                  range=[400,800],
                                  domain=yd[3],
                                  showgrid=False
                                 ), #far field
                      xaxis10=dict(anchor='y10',
                                   matches='x12',
                                   showticklabels=False,
                                   domain=[w[1],w[-2]]
                                  ),
                      yaxis10=dict(title='Position NF (µm)',
                                   titlefont=dict(color=color_Position[0]),
                                   tickfont=dict(color=color_Position[0]),
                                   domain=yd[4],
                                   range=[-200,200],
                                   showgrid=False
                                  ), #near field
                      yaxis11=dict(title='Width NF (mm)',
                                   titlefont=dict(color=color_Width[0]),
                                   tickfont=dict(color=color_Width[0]),
                                   anchor='x11',
                                   overlaying='y10',
                                   matches=None,
                                   side='right',
                                   range=[8,11],
                                   domain=yd[4],
                                   showgrid=False
                                  ), #near field
                      xaxis12=dict(anchor='y12',
                                   showticklabels=True,
                                   domain=[w[1],w[-2]],
                                   tickangle=45
                                  ), #common x shared axis
                      yaxis12=dict(title='Flow (L/min)',
                                   anchor='x12',
                                   overlaying='y12',
                                   matches=None,
                                   side='left',
                                   range=[0,10],
                                   domain=yd[5],
                                   showgrid=False
                                  ) #flow
                     )
    
    ### UV ###
    #Energy
    for i in range(len(df_UV.columns)):
        fig.add_trace(go.Scatter(x=df_UV.index,
                                 y=df_UV[df_UV.columns[i]],
                                 name=('%s' %(df_UV.columns[i])),
                                 xaxis='x1',
                                 yaxis='y1',
                                 line=dict(color=color_UV[i]),
                                 visible=True,
                                 legendgroup='UV',
                                 legendgrouptitle_text='UV'
                                )
                     )
    #Mean, Max, Min
    for i in range(len(df_UV_hide.columns)):
        fig.add_trace(go.Scatter(x=df_UV_hide.index,
                                 y=df_UV_hide[df_UV_hide.columns[i]],
                                 name=('%s' %(df_UV_hide.columns[i])),
                                 xaxis='x1',
                                 yaxis='y1',
                                 line=dict(color=color_UV_hide[i]),
                                 visible='legendonly',
                                 legendgroup='UV'
                                )
                     )

    #RMS
    fig.add_trace(go.Scatter(x=df_UV_RMS.index,
                             y=df_UV_RMS[df_UV_RMS.columns[0]],
                             name='UV RMS',
                             xaxis='x1',
                             yaxis='y2',
                             line=dict(color=color_UV_RMS[0]),
                             visible=True,
                             legendgroup='UV'
                            )
                 )

    #Power
    for i in range(len(df_P.columns)):
        fig.add_trace(go.Scatter(x=df_P.index,
                                 y=df_P[df_P.columns[i]],
                                 name=('%s' %(df_P.columns[i])),
                                 xaxis='x1',
                                 yaxis='y3',
                                 line=dict(color=color_P[i]),
                                 visible=True,
                                 legendgroup='UV'
                                )
                     )

    ### IR ###
    #Energy
    for i in range(len(df_IR.columns)):
        fig.add_trace(go.Scatter(x=df_IR.index,
                                 y=df_IR[df_IR.columns[i]],
                                 name=('%s' %(df_IR.columns[i])),
                                 xaxis='x4',
                                 yaxis='y4',
                                 line=dict(color=color_IR[i]),
                                 visible=True,
                                 legendgroup='IR',
                                 legendgrouptitle_text='IR'
                                )
                     )

    #Mean, Max, Min
    for i in range(len(df_IR_hide.columns)):
        fig.add_trace(go.Scatter(x=df_IR_hide.index,
                                 y=df_IR_hide[df_IR_hide.columns[i]],
                                 name=('%s' %(df_IR_hide.columns[i])),
                                 xaxis='x4',
                                 yaxis='y4',
                                 line=dict(color=color_IR_hide[i]),
                                 visible='legendonly',
                                 legendgroup='IR'
                                )
                     )

    #RMS
    fig.add_trace(go.Scatter(x=df_IR_RMS.index,
                             y=df_IR_RMS[df_IR_RMS.columns[0]],
                             name='IR RMS',
                             xaxis='x4',
                             yaxis='y5',
                             line=dict(color=color_IR_RMS[0]),
                             visible=True,
                             legendgroup='IR'
                            )
                 )
    
    #Depolar power
    for i in range(len(df_Pd.columns)):
        fig.add_trace(go.Scatter(x=df_Pd.index,
                                 y=df_Pd[df_Pd.columns[i]],
                                 name=('%s' %(df_Pd.columns[i])),
                                 xaxis='x4',
                                 yaxis='y6',
                                 line=dict(color=color_Pd[i]),
                                 visible=True,
                                 legendgroup='IR'
                                )
                     )
        
    ### TEMPERATURE ###
    #Flowmeter
    for i in range(len(df_flow_tmp.columns)):
        fig.add_trace(go.Scatter(x=df_flow_tmp.index,
                                 y=df_flow_tmp[df_flow_tmp.columns[i]],
                                 name=('%s' %(df_flow_tmp.columns[i])),
                                 xaxis='x7',
                                 yaxis='y7',
                                 visible=True,
                                 legendgroup='Temperature',
                                 legendgrouptitle_text='Temperature'
                                )
                     )
    
    #Watlow
    for i in range(len(df_watlow.columns)):
        fig.add_trace(go.Scatter(x=df_watlow.index,
                                 y=df_watlow[df_watlow.columns[i]],
                                 name=('%s' %(df_watlow.columns[i])),
                                 xaxis='x7',
                                 yaxis='y7',
                                 visible=True,
                                 legendgroup='Temperature'
                                )
                     )
    
    #Picolog
    for i in range(len(df_pico.columns)):
        fig.add_trace(go.Scatter(x=df_pico.index,
                                 y=df_pico[df_pico.columns[i]],
                                 name=('%s' %(df_pico.columns[i])),
                                 xaxis='x7',
                                 yaxis='y7',
                                 visible='legendonly',
                                 legendgroup='Temperature'
                                )
                     )

    ### FAR FIELD ###
    #Position x
    fig.add_trace(go.Scatter(x=df_farfield.index, 
                             y=df_farfield['Position x FF'], 
                             name='Position x - Far Field',
                             xaxis='x8',
                             yaxis='y8',
                             line=dict(color=color_Position[0]),
                             visible=True,
                             legendgroup='Far Field',
                             legendgrouptitle_text='Far Field',
                            )
                 )
    
    #Position y
    fig.add_trace(go.Scatter(x=df_farfield.index, 
                             y=df_farfield['Position y FF'], 
                             name='Position y - Far Field',
                             xaxis='x8',
                             yaxis='y8',
                             line=dict(color=color_Position[1]),
                             visible=True,
                             legendgroup='Far Field'
                            )
                 )
    
    #Width x
    fig.add_trace(go.Scatter(x=df_farfield.index, 
                             y=df_farfield['1/e^2 X width FF'], 
                             name='1/e^2 x - Far Field',
                             xaxis='x8',
                             yaxis='y9',
                             line=dict(color=color_Width[0]),
                             visible=True,
                             legendgroup='Far Field'
                            )
                 )
    
    #Width y
    fig.add_trace(go.Scatter(x=df_farfield.index, 
                             y=df_farfield['1/e^2 Y width FF'], 
                             name='1/e^2 y - Far Field',
                             xaxis='x8',
                             yaxis='y9',
                             line=dict(color=color_Width[1]),
                             visible=True,
                             legendgroup='Far Field'
                            )
                 ) 
    
    ### NEAR FIELD ###
    #Position x
    fig.add_trace(go.Scatter(x=df_nearfield.index, 
                             y=df_nearfield['Position x NF'], 
                             name='Position x - Near Field',
                             xaxis='x10',
                             yaxis='y10',
                             line=dict(color=color_Position[0]),
                             visible=True,
                             legendgroup='Near Field',
                             legendgrouptitle_text='Near Field',
                            )
                 )
    
    #Position y
    fig.add_trace(go.Scatter(x=df_nearfield.index, 
                             y=df_nearfield['Position y NF'], 
                             name='Position y - Near Field',
                             xaxis='x10',
                             yaxis='y10',
                             line=dict(color=color_Position[1]),
                             visible=True,
                             legendgroup='Near Field'
                            )
                 )
    
    #Width x
    fig.add_trace(go.Scatter(x=df_nearfield.index, 
                             y=df_nearfield['1/e^2 X width NF'], 
                             name='1/e^2 x - Near Field',
                             xaxis='x10',
                             yaxis='y11',
                             line=dict(color=color_Width[0]),
                             visible=True,
                             legendgroup='Near Field'
                            )
                 )
    
    #Width y
    fig.add_trace(go.Scatter(x=df_nearfield.index, 
                             y=df_nearfield['1/e^2 Y width NF'], 
                             name='1/e^2 y - Near Field',
                             xaxis='x10',
                             yaxis='y11',
                             line=dict(color=color_Width[1]),
                             visible=True,
                             legendgroup='Near Field'
                            )
                 )
    
    ### FLOW ###
    for i in range(len(df_flow.columns)):
        fig.add_trace(go.Scatter(x=df_flow.index,
                                 y=df_flow[df_flow.columns[i]],
                                 name=('%s' %(df_flow.columns[i])),
                                 xaxis='x12',
                                 yaxis='y12',
                                 visible=True,
                                 legendgroup='Flow',
                                 legendgrouptitle_text='Flow'
                                )
                     )
        
    # fig.show()
    fig.write_html(path_result, 
                   auto_open=True,
                   config={'modeBarButtonsToAdd':['drawline',
                                                  'drawopenpath',
                                                  'drawclosedpath',
                                                  'drawcircle',
                                                  'drawrect',
                                                  'eraseshape'
                                                 ]
                          }
                  )
    
    return 1

# Température #

In [12]:
data_Flowmeter_tmp = extract_data_flowmeter(path_Flowmeter)
data_Flowmeter_tmp.drop(columns = ['Bench flow', 'Oscillator flow', 'Amplifier flow'], inplace = True)
data_Flowmeter_tmp.rename(columns = {'Bench temperature' : 'Bench',
                                     'Oscillator temperature' : 'Oscillator',
                                     'Amplifier temperature' : 'Amplifier'
                                    },
                          inplace=True
                         )
data_Watlow = extract_data_watlow(path_Watlow)
data_Picolog = extact_data_picolog(path_Picolog)

plot_temperature(data_Flowmeter_tmp, data_Watlow, data_Picolog[::sampling_rate])

1

# Débit #

In [13]:
data_Flowmeter = extract_data_flowmeter(path_Flowmeter)
data_Flowmeter.drop(columns = ['Bench flowmeter', 'Oscillator flowmeter', 'Amplifier flowmeter'], inplace = True)
data_Flowmeter.rename(columns = {'Bench flow' : 'Bench',
                                 'Oscillator flow' : 'Oscillator',
                                 'Amplifier flow' : 'Amplifier'
                                 },
                       inplace=True
                      )

plot_flow(data_Flowmeter)

1

# UV #

In [23]:
data_UV = extract_data_UV(path_Ulink)
data_UV_stat = extract_data_UV_stat(path_Ulink)
data_UV_RMS = pd.DataFrame(data_UV_stat['RMS UV'])
data_UV_hide = data_UV_stat.drop(columns = ['Sigma UV', 'RMS UV'], inplace = False)

data_P = extract_data_P(path_P)

plot_UV(data_UV, data_UV_RMS, data_UV_hide, data_P)

1

# IR #

In [15]:
data_IR = extract_data_IR(path_Ulink)
data_IR_stat = extract_data_IR_stat(path_Ulink)
data_IR_RMS = pd.DataFrame(data_IR_stat['RMS IR'])
data_IR_hide = data_IR_stat.drop(columns = ['Sigma IR', 'RMS IR'], inplace = False)

data_Puissance_depolar = extract_data_Puissance_depolar(path_Ulink)

plot_IR(data_IR, data_IR_RMS, data_IR_hide, data_Puissance_depolar)

1

# Caméra - Champ proche #

In [16]:
data_NearField = extract_data_cameraNF(path_NearField)
plot_field(data_NearField, field='Near')

1

# Caméra - Champ lointain #

In [17]:
data_FarField = extract_data_cameraFF(path_FarField)
plot_field(data_FarField, field='Far')

1

In [18]:
dataframes = [data_Flowmeter_tmp[::sampling_rate], 
              data_Watlow[::sampling_rate], 
              data_Picolog[::2*sampling_rate], 
              data_Flowmeter[::sampling_rate], 
              data_UV[::sampling_rate], 
              data_UV_RMS[::(int(sampling_rate/12)+1)], 
              data_UV_hide[::(int(sampling_rate/12)+1)], 
              data_P[::sampling_rate], data_IR[::sampling_rate], 
              data_IR_RMS[::(int(sampling_rate/12)+1)], 
              data_IR_hide[::(int(sampling_rate/12)+1)], 
              data_Puissance_depolar[::sampling_rate], 
              data_NearField[::sampling_rate], 
              data_FarField[::sampling_rate]
             ]
plot_xshared(dataframes)

1

In [19]:
df = pd.DataFrame()
for i in range (len(dataframes)):
    df = pd.merge(df, dataframes[i], left_index=True, right_index=True, how='outer')

In [20]:
fig = px.imshow(df.corr(method='pearson'), 
                text_auto=False,
                #contrast_rescaling='infer',
                #aspect="auto"
               )
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.update_layout(dragmode='drawrect',
                  newshape=dict(line_color='cyan')
                 )
# fig.show()
fig.write_html(TGI_path + '\heatmap.html',
               auto_open=True,
               config={'modeBarButtonsToAdd':['drawline',
                                              'drawopenpath',
                                              'drawclosedpath',
                                              'drawcircle',
                                              'drawrect',
                                              'eraseshape'
                                             ]
                      }
              )